# Validation Sets e Test Sets

L'esercizio di Colab precedente ha valutato il modello allenato, usando i dati presenti nel training set, che non fornisce un forte segnale riguardo la qualità del modello. In questo colab, sperienterai con il validation set e il test set.





## Il dataset

Come nell'esercizio precedente, questo esercizio usa il [California Housing dataset](https://developers.google.com/machine-learning/crash-course/california-housing-data-description) per predire `median_house_value` per isolato. Come per molti dataset "famosi", il dataset California Housing consiste in realtà di due dataset separati, ognuno salvato in un .csv diverso:

* Il training set situato nel file `california_housing_train.csv`.
* Il test set situato nel file `california_housing_test.csv`.

Creiamo il validation set dividendo il training set che abbiamo scaricato in due parti:

* un training set più piccolo
* e un validation set

## Uso della versione aggiornata di TensorFlow


In [ ]:
#@title EseguiTensorFlow 2.x
%tensorflow_version 2.x

## Import di moduli importanti


In [ ]:
#@title Importa i moduli
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

## Caricamento dei dataset da internet

Il codice seguente carica i file .csv separati e crea i seguenti due DataFrames pandas:
* `train_df`, che contiene il training set.
* `test_df`, che contiene il test set.



In [ ]:
train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")

## Scala i valori delle lables

Il seguente codice scala `median_house_value`.

In [ ]:
scale_factor = 1000.0

# Scala la label del training set
train_df["median_house_value"] /= scale_factor 

# Scala la label del test set
test_df["median_house_value"] /= scale_factor

## Caricamento delle funzioni per costruire ed allenare il modello 

Il seguente codice definisce due funzioni:
* `build_model`, che definisce la topografia del modello.
* `train_model`, che allena il modello, dando in output non solo il valore di loss per il training set ma anche il valore di loss per il validation set.


In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model               


def train_model(model, df, feature, label, my_epochs, 
                my_batch_size=None, my_validation_split=0.1):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=my_validation_split)

  # Gather the model's trained weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
  epochs = history.epoch
  
  # Isolate the root mean squared error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history   

print("Defined the build_model and train_model functions.")

## Definizione delle funzioni per disegnare i grafici

La funzione `plot_the_loss_curve`disegna la loss vs. epoche sia per il training set sia per il validations set.

In [ ]:
#@title Definizione della funzione di plot

def plot_the_loss_curve(epochs, mae_training, mae_validation):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs[1:], mae_training[1:], label="Training Loss")
  plt.plot(epochs[1:], mae_validation[1:], label="Validation Loss")
  plt.legend()
  
  # We're not going to plot the first epoch, since the loss on the first epoch
  # is often substantially greater than the loss for other epochs.
  merged_mae_lists = mae_training[1:] + mae_validation[1:]
  highest_loss = max(merged_mae_lists)
  lowest_loss = min(merged_mae_lists)
  delta = highest_loss - lowest_loss
  print(delta)

  top_of_y_axis = highest_loss + (delta * 0.05)
  bottom_of_y_axis = lowest_loss - (delta * 0.05)
   
  plt.ylim([bottom_of_y_axis, top_of_y_axis])
  plt.show()  

print("Defined the plot_the_loss_curve function.")

## Task 1: Sperimenta con lo splitting del validation set

Nel seguente codice, vedrai una variabile chiamata `validation_split`, che abbiamo inizializzato a 0.2. La variabile `validation_split` specifica la proporzione del training set originale che servirà da validation set. Il training set originale contiene 17,000 dati. Di conseguenza, un `validation_split` di 0.2 significa che:
* 17,000 * 0.2 ~= 3,400 dati saranno nel validation set.
* 17,000 * 0.8 ~= 13,600 dati saranno nel training set.

Il seguente codice costruisce un modello, lo allena sul training set, e valuta il modello costruito su entrambi i :
* validation set
* training set
Se i dati nel training set sono simili ai dati nel validation set, allora le due curve di loss e la loss finale dovrebbe essere quasi identica. Tuttavia, le curve di loss e il valore di loss finale **non** sono quasi identici. Hmmm,Questo è strano.

Sperimenta con due o tre valori differenti per `validation_split`. Diversi valori di `validation_split` risolvono il problema?



In [ ]:
# Le seguenti variabili sono gli iperparametri.
learning_rate = 0.08
epochs = 30
batch_size = 100

# Dividi il training set originale in un training set più piccolo e un validation set
validation_split = 0.6

# Identifica la feature e la label
my_feature = "median_income"    # Il reddito medio in uno specifico isolato.
my_label = "median_house_value" # Il valore medio di una casa in uno specifico isolato.

# Ovvero, stiamo andando a creare un modello che predice il valore di una casa
# basandosi soltanto sul reddito medio del quartiere

# Ci sbarazziamo di qualsiasi versione pre-esistente del modello.
my_model = None

# Invoca le funzioni di costruzione ed allenamento del modello.
my_model = build_model(learning_rate)
epochs, rmse, history = train_model(my_model, train_df, my_feature, 
                                    my_label, epochs, batch_size, 
                                    validation_split)

plot_the_loss_curve(epochs, history["root_mean_squared_error"], 
                    history["val_root_mean_squared_error"])

## Task 2: Determina **perché** le curve di loss differiscono

Non importa come dividi il training set e validation set, le curve di loss differiscono significativamente. Evidentemente, i dati nel training set non sono abbastanza simili ai dati nel validation set. Controintuitivo?Si, ma questo problema è abbastanza comune nel machine learning.

Il tuo compito è determinare **perché** le curve di loss non sono molto simili. Come per molti problemi nel machine learning, il problema è radicato nei dati stessi. Per risolvere questo mistero del perché il training set e il validation set non sono quasi identici, scrivi una linea o due di [codice pandas](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb?utm_source=validation-colab&utm_medium=colab&utm_campaign=colab-external&utm_content=pandas_tf2-colab&hl=en) qui di seguito. Qui ci sono dei suggerimenti:

* Il codice precedente splittava il training set originale in:
  * un training set più piccolo (il training set originale - validation set)
  * il validation set
* Di default, il metodo di pandas [`head`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) da in output le *prime* 5 righe del Dataframe. Per vedere di più del training set, specifica la `n` come argomento di `head` ed assegna un numero positvo. 

In [ ]:
# Write some code in this code cell.
train_df.head(10)

In [ ]:
#@title Possibile risposta.

# Esamina gli esempi da 0 a 4 e gli esempi da 25 a 29 del training set
train_df.head(n=1000)

# Il training set originale è ordinato per longitudine.
# Apparentemente, la longitudine influenza la relazione tra
# total_rooms e il median_house_value.


## Task 3. Risoluzione del problema

Per risolvere il problema, mischia i dati nel training set prima di effettuare lo splitting in training set e validation set. Per fare questo, segui i seguenti step:

1. Mischia i dati nel training set aggiungendo la seguente lina in qualsiasi punto prima della chiamata a `train_model`


```
  shuffled_train_df = train_df.reindex(np.random.permutation(train_df.index))
```                                    

2. Passa `shuffled_train_df` (invece di `train_df`) come secondo argomento a `train_model` in modo che la chiamata diventi:

```
  epochs, rmse, history = train_model(my_model, shuffled_train_df, my_feature, 
                                      my_label, epochs, batch_size, 
                                      validation_split)
```

In [ ]:
#@title Double-click to view the complete implementation.

# The following variables are the hyperparameters.
learning_rate = 0.08
epochs = 70
batch_size = 100

# Split the original training set into a reduced training set and a
# validation set. 
validation_split = 0.2

# Identify the feature and the label.
my_feature = "median_income"    # the median income on a specific city block.
my_label = "median_house_value" # the median house value on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on the neighborhood's median income.  

# Discard any pre-existing version of the model.
my_model = None

# Shuffle the examples.
shuffled_train_df = train_df.reindex(np.random.permutation(train_df.index)) 

# Invoke the functions to build and train the model. Train on the shuffled
# training set.
my_model = build_model(learning_rate)
epochs, rmse, history = train_model(my_model, shuffled_train_df, my_feature, 
                                    my_label, epochs, batch_size, 
                                    validation_split)

plot_the_loss_curve(epochs, history["root_mean_squared_error"], 
                    history["val_root_mean_squared_error"])

Sperimenta con `validation_split` per rispondere alle seguenti domande:

* Con il training set mischiato, la loss finale per il training set è più vicina alla loss finale del validation set?
* Con quale range di valori di `validation_split` il valore della loss finale per training set e validation set divergono vistosamente? Perché?

In [ ]:
#@title Risposte

''' Si, dopo aver mischiato il training set originale, la loss finale per il training
set e il validation set sono diventate molto più simili.'''

'''Se la validation_split <0.15 i valori della loss finale per il training
set e il validation set divergeranno significativamente. Apparentemente,
il validation set non contiene più dati a sufficienza.'''


## Task 4: Use the Test Dataset to Evaluate Your Model's Performance
## Task 4: Usa il dataset di test per  valutare le performance del modello

Il test set di solito funge come ultimo giudizio per la qualità del modello. Il test set può servire da giudice imparziale perché i suoi dati non sono stati usati in fase di allenamento del modello. Esegui il codice sottostante per valutare il modello con il test set:

In [ ]:
x_test = test_df[my_feature]
y_test = test_df[my_label]

results = my_model.evaluate(x_test, y_test, batch_size=batch_size)

Compara la radice dell'errore quadratico medio del modello quando valutato su ciascuno dei tre dataset:
* training set: cerca `root_mean_squared_error` nell'epoca finale di training.
* validaion set: cerca `val_root_mean_squared_error` nella epoca finale di training
* test set: esegui il codice precedente ed esamina `root_mean_squared_error`.

Idealmente, la root mean squared error di tutte e tre dovrebbe essere simile. Lo sono?


In [ ]:
#@title Risposta

'''Nei nostri esperimenti, si, i valori di 
rmse sono simili abbastanza. '''